![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)


<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Finteresting-problems&branch=main&subPath=notebooks/hundred-prisoners-problem.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Hundred Prisoners Problem

"The 100 prisoners problem is a mathematical problem where 100 numbered prisoners must find their own numbers in one of 100 drawers in order to survive... each prisoner may open only 50 drawers and cannot communicate with other prisoners." [Wikipedia - 100 prisoners problem](https://en.wikipedia.org/wiki/100_prisoners_problem)

[Veritasium](https://www.veritasium.com) posted a [video description and solution](https://www.youtube.com/watch?v=iSNsgj1OCLA) to this problem. [MinutePhysics](https://www.minutephysics.com/) also posted a [similar video](https://www.youtube.com/watch?v=eivGlBKlK6M) and [solution](https://www.youtube.com/watch?v=C5-I0bAuEUE), as did [Stand-up Maths](https://www.youtube.com/watch?v=a1DUUnhk3uE). In this notebook we'll follow Veritasium's calculations.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('iSNsgj1OCLA')

Calculating the probability of success with random choices

In [ ]:
(1/2) ** 100

Finding the total number of unique loops

In [ ]:
from math import factorial
factorial(100)/100

Probability of a loop with a length of 100

In [ ]:
#(factorial(100)/100)/factorial(100)
1/100

Probability of a loop longer than 50

In [ ]:
total = 0
for x in range(50, 100):
    total += 1/x
total

Probability of a loop 50 or shorter

In [ ]:
1 - total

Probability distribution visualization

In [ ]:
import functools
from math import floor, factorial
from numpy import nan
import pandas as pd
import plotly.express as px

# thank you to Zachary Shand
@functools.cache
def perm_count(k,n):
    if k == 1:
        return 1
    s = 0
    lim1 = floor(n/k)
    for j in range(1,lim1+1):
        term1 = 1/factorial(j)/k**j*factorial(n)/factorial(n-k*j)
        lim2 = min(k-1, n-j*k)
        term2 = 0
        for t in range(1,lim2+1):
            term2 += perm_count(t, n-k*j)
        if term2 == 0:
            term2 = 1
        s += term1*term2
    return s

n = 100
perm_counts = {i:perm_count(i,n) for i in range(1,n+1)}
perm_probs = {key:val/factorial(n) for key, val in perm_counts.items()}
x = list(range(1,n+1))
reciprocal = [nan if xi < n/2 else 1/xi for xi in x]

df = pd.DataFrame(columns=['Longest Loop', 'Probability', 'Outcome'])
df['Longest Loop'] = x
df['Probability'] = [perm_probs[xi] for xi in x]
df.loc[df['Longest Loop'] > n/2, 'Outcome'] = 'Failure'
df.loc[df['Longest Loop'] <= n/2, 'Outcome'] = 'Success'

success_rate = df.loc[df['Outcome'] == 'Success', 'Probability'].sum()
failure_rate = df.loc[df['Outcome'] == 'Failure', 'Probability'].sum()

fig = px.bar(df, x='Longest Loop', y='Probability', color='Outcome', title='Probabilities of Longest Loop Lengths')
fig.add_annotation(x=20, y=0.017, showarrow=False, text=success_rate)
fig.add_annotation(x=80, y=0.017, showarrow=False, text=failure_rate)
fig.show()

Comparing the probabilities of prisoners randomly finding their number to using the loop strategy

In [ ]:
iterations = 10000
prisoners = 100

from random import choice
from collections import Counter
import plotly.graph_objects as go
fig2 = go.Figure()

# simulate random strategy
list_of_successes = []
for i in range(iterations):
    successes = 0
    for p in range(prisoners):
        if choice([True, False]):
            successes += 1          # add 1 if the prisoner is successful
    list_of_successes.append(successes)
c = Counter(list_of_successes)
y = [k/iterations for k in list(c.values())]   # divide the number of successes by the number of iterations
fig2.add_trace(go.Bar(x=list(c.keys()), y=y, marker=dict(color='green'), name='Random Strategy'))

# loop strategy from previous calculations
successes = df.loc[df['Outcome'] == 'Success', 'Probability'].to_dict()
successes[100] = success_rate
fig2.add_trace(go.Bar(x=list(successes.keys()), y=list(successes.values()), marker=dict(color='blue'), name='Loop Strategy'))

fig2.update_xaxes(title_text='Number of Prisoners who Succeed')
fig2.update_yaxes(title_text='Probability')
fig2.update_layout(title='Probabilities of Success: Random and Loop Strategies')
fig2.add_annotation(x=25, y=0.05, showarrow=False, text='{:.3f}'.format(failure_rate), font={'size':22,'color':'blue'})
fig2.show()

Calculating probabilities of success with more prisoners

In [ ]:
def success_rate(prisoners):
    denominators = list(range(int(prisoners/2)+1, prisoners+1))
    successes = 1 - sum([1/d for d in denominators])
    print(prisoners, ':', successes*100, '%')

success_rate(100)
success_rate(1000)
success_rate(1000000)

Calculating the probability of sucess as the number of prisoners goes to infinity as **one** minus **the area under the curve**, or $1 - \ln 2$, since $\ln 2$ would give us the probability of failure.

In [ ]:
from math import log
1 - log(2)

So we see that even with a number of prisoners approaching infinity, there is still more than a 30% chance of every prisoner succeeding using this loop strategy.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)